In [1]:
import pandas as pd
from fredapi import Fred
import sys
import os
import hvplot.pandas as hvplot

# Add config module to sys path
sys.path.insert(0, os.path.abspath('../../config'))

# Import API key
from api_keys import fred_api_key


In [2]:
# Set up API Key

fred = Fred(api_key=f'{fred_api_key}')

In [3]:
# Retrieve 10 Year Treasury security rates (monthly) from FRED API

data_10year_treasury = fred.get_series('GS10')

In [4]:
# Retain only data since Jan 1959

data_10year_treasury = data_10year_treasury.loc[data_10year_treasury.index > '1958-12-31']

In [5]:
# View the 10 year Series
data_10year_treasury.head(5)

1959-01-01    4.02
1959-02-01    3.96
1959-03-01    3.99
1959-04-01    4.12
1959-05-01    4.31
dtype: float64

In [6]:
# Retrieve 2 Year Treasury security rates (monthly) from FRED API

data_2year_treasury = fred.get_series('GS2')

In [7]:
# View the 2 year Series

data_2year_treasury.head()

1976-06-01    7.06
1976-07-01    6.85
1976-08-01    6.63
1976-09-01    6.42
1976-10-01    5.98
dtype: float64

In [8]:
# Retrieve 1 Year Treasury security rates (monthly) from FRED API

data_1year_treasury = fred.get_series('GS1')

In [9]:
# Retain only data since Jan 1959

data_1year_treasury = data_1year_treasury.loc[data_1year_treasury.index > '1958-12-31']

In [10]:
# View the 1 year Series

data_1year_treasury.head()

1959-01-01    3.36
1959-02-01    3.54
1959-03-01    3.61
1959-04-01    3.72
1959-05-01    3.96
dtype: float64

In [11]:
# Create DataFrame for each Series downloaded from Fred API

data_10year_treasury_df = pd.DataFrame(data_10year_treasury)

data_2year_treasury_df = pd.DataFrame(data_2year_treasury)

data_1year_treasury_df = pd.DataFrame(data_1year_treasury)


In [12]:
# Reset Index of 10 Year DataFrame

data_10year_treasury_df = data_10year_treasury_df.reset_index()

In [13]:
# View 10 Year DataFrame

data_10year_treasury_df.head()

,index,0
0,1959-01-01,4.02
1,1959-02-01,3.96
2,1959-03-01,3.99
3,1959-04-01,4.12
4,1959-05-01,4.31


In [14]:
# Reset index of 2 year DataFrame

data_2year_treasury_df = data_2year_treasury_df.reset_index()

In [15]:
# View 2 Year DataFrame

data_2year_treasury_df.head()

,index,0
0,1976-06-01,7.06
1,1976-07-01,6.85
2,1976-08-01,6.63
3,1976-09-01,6.42
4,1976-10-01,5.98


In [16]:
# Reset index of 1 year DataFrame 

data_1year_treasury_df = data_1year_treasury_df.reset_index()

In [17]:
# View 1 year DataFrame

data_1year_treasury_df.head()

,index,0
0,1959-01-01,3.36
1,1959-02-01,3.54
2,1959-03-01,3.61
3,1959-04-01,3.72
4,1959-05-01,3.96


In [18]:
# Merge 10 Year and 2 Year DataFrame to create Treasury DataFrame

treasury_df = pd.merge(data_10year_treasury_df,data_2year_treasury_df, on="index", how="left")

In [19]:
# Fill 0 for missing values

treasury_df = treasury_df.fillna(0)

In [20]:
# View the Treasury DataFrame

treasury_df.head()

,index,0_x,0_y
0,1959-01-01,4.02,0.0
1,1959-02-01,3.96,0.0
2,1959-03-01,3.99,0.0
3,1959-04-01,4.12,0.0
4,1959-05-01,4.31,0.0


In [21]:
# Merge Treasury DataFrame  and 1 Year Data Frame

treasury_df = pd.merge(treasury_df, data_1year_treasury_df, on="index")

In [22]:
# View the Treasury DataFrame

treasury_df.head()

,index,0_x,0_y,0
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [23]:
# Rename columns of Treasury DataFrame 

treasury_df = treasury_df.rename(columns={"index":"date", 
                                          "0_x":"10 Years T Rates", 
                                          "0_y": "2 Years T Rates",
                                         0 : "1 Year T Rates"})

In [24]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates
0,1959-01-01,4.02,0.0,3.36
1,1959-02-01,3.96,0.0,3.54
2,1959-03-01,3.99,0.0,3.61
3,1959-04-01,4.12,0.0,3.72
4,1959-05-01,4.31,0.0,3.96


In [25]:
# Create a function to fill missing 2 Year Rates with 10 Year rates.
# This will make the rate differnece zero for the period data for 2 year rates are missing

def adjust_2_year_missing_rates(twoYearValue, tenYearValue):
    if twoYearValue == 0:
        return tenYearValue
    else:
        return twoYearValue

In [26]:
# Adjust missing data for 2 year treasury

treasury_df["2 Years T Rates(Adj)"] = treasury_df.apply(lambda row :  
                                                   adjust_2_year_missing_rates(row["2 Years T Rates"], 
                                                                               row["10 Years T Rates"]), 
                                                   axis=1)

In [27]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,2 Years T Rates(Adj)
0,1959-01-01,4.02,0.0,3.36,4.02
1,1959-02-01,3.96,0.0,3.54,3.96
2,1959-03-01,3.99,0.0,3.61,3.99
3,1959-04-01,4.12,0.0,3.72,4.12
4,1959-05-01,4.31,0.0,3.96,4.31


In [28]:
# Create 10 - 2 years Rate Difference column

treasury_df["10-2 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["2 Years T Rates(Adj)"]

In [29]:
# Create 10 - 1 year Rate Difference column

treasury_df["10-1 Rate Diff"] = treasury_df["10 Years T Rates"] - treasury_df["1 Year T Rates"]

In [30]:
# View the Treasury DataFrame

treasury_df.head()

,date,10 Years T Rates,2 Years T Rates,1 Year T Rates,2 Years T Rates(Adj),10-2 Rate Diff,10-1 Rate Diff
0,1959-01-01,4.02,0.0,3.36,4.02,0.0,0.66
1,1959-02-01,3.96,0.0,3.54,3.96,0.0,0.42
2,1959-03-01,3.99,0.0,3.61,3.99,0.0,0.38
3,1959-04-01,4.12,0.0,3.72,4.12,0.0,0.40
4,1959-05-01,4.31,0.0,3.96,4.31,0.0,0.35


In [31]:
# Chart Treasury Securities rates for 10 Years 

plot_10_years_rates = treasury_df.hvplot.line(x="date", y="10 Years T Rates", 
                                              width = 300, title = "Treasury - 10 Years Rates")

In [32]:
# Chart Treasury Securities rates for 2 Years 

plot_2_years_rates = treasury_df.hvplot.line(x="date", y="2 Years T Rates", 
                                             width = 300, title = "Treasury - 2 Years Rates")


In [33]:
# Chart Treasury Securities rates for 1 Year

plot_1_year_rates = treasury_df.hvplot.line(x="date", y="1 Year T Rates",
                                            width = 300, title = "Treasury - 1 Years Rates")


In [34]:
# Display Treasury Securities rates for 10 Years, 2 years and 1 year

plot_10_years_rates + plot_2_years_rates + plot_1_year_rates


:Layout
   .Curve.I   :Curve   [date]   (10 Years T Rates)
   .Curve.II  :Curve   [date]   (2 Years T Rates)
   .Curve.III :Curve   [date]   (1 Year T Rates)

In [35]:
# Create a plot to chart the Interest rate difference between 10 years and 2 years Treasury

treasury_df.hvplot.line(x="date", y="10-2 Rate Diff", title = "Treasury Securities - 10 Years Minus 2 Years Interest Rate Difference")


:Curve   [date]   (10-2 Rate Diff)

In [36]:
# Create a plot to chart the Interest rate difference between 10 years and 1 year Treasury

treasury_df.hvplot.line(x="date", y="10-1 Rate Diff", title = "Treasury Securities - 10 Years Minus 1 Year Interest Rate Difference")

:Curve   [date]   (10-1 Rate Diff)